In [135]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import _LRScheduler

import numpy as np
import pandas as pd
import os
import glob
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pathlib import Path

Dataset and Dataloaders

In [136]:


class TimeSeriesDataset(Dataset):
    def __init__(self, x_set, y_set, seq_len):
        self.x, self.y = x_set, y_set
        self.seq_len = seq_len

    def __len__(self):
        # return int(np.ceil(len(self.x) / float(self.seq_len)))
        return int((len(self.x) ))


    def __getitem__(self, idx):
        # start_idx = idx * self.seq_len
        # end_idx = (idx + 1) * self.seq_len
        
        # sliding window
        start_idx = idx
        end_idx = idx + self.seq_len

        batch_x = self.x[start_idx+1:end_idx+1]
        batch_y = self.y[start_idx:end_idx+1]

        # Count the occurrences of each row
        unique_rows, counts = np.unique(batch_y, axis=0, return_counts=True)
        
        # print("index:",idx,"ybatch:",batch_y)

        # Get the index of the row with the highest count
        most_common_row_index = np.argmax(counts)

        # Get the most common row
        most_common_row = unique_rows[most_common_row_index]
        
        
        batch_y_bin = most_common_row

        # Convert NumPy arrays to PyTorch tensors
        batch_x = torch.from_numpy(batch_x)
        batch_y = torch.from_numpy(batch_y)
        batch_y_bin = torch.from_numpy(batch_y_bin)

        # Pad sequences to ensure they have the same length within the batch
        pad_len = self.seq_len - batch_x.shape[0]
        if pad_len > 0:
            pad_shape = (pad_len,) + batch_x.shape[1:]
            pad_shape_y = (pad_len,) + batch_y.shape[1:]

            batch_x = torch.cat([batch_x, torch.zeros(pad_shape)], dim=0)
            batch_y = torch.cat([batch_y, torch.zeros(pad_shape_y)], dim=0)

        return batch_x, batch_y, batch_y_bin

    def on_epoch_end(self):
        indices = np.arange(len(self.x))
        np.random.shuffle(indices)
        self.x = self.x[indices]
        self.y = self.y[indices]


In [137]:
# dataloaders
def generate_data(features, batch_size, seq_len):    
    
    csv_path = './processed_data'
    
    csv_files = glob.iglob(csv_path + "/**/*.csv", recursive=True)
    
    df_list = []
    

    for file in csv_files:
        df_list.append(pd.read_csv(file))
            

    print('All Trials: ',len(df_list))
    
#     # Concatenate all DataFrames
    df = pd.concat(df_list, ignore_index=True)

    df = df.drop(df.columns[0], axis=1)
    
    lb = preprocessing.LabelBinarizer()

    df_labels= df.pop('label')
    df_features = df
    
    # print(df_labels)
    # df_features = df_features['landmarks'].apply(pd.Series)


#     test_labels= test_df.pop('label')
#     test_features = test_df


    all_interventions = ["I0", 'I1', 'I2']
    lb.fit(all_interventions)
    # lb.fit(df_labels)
    
    print(lb.classes_)
    


    df_labels = lb.transform(df_labels)
#     test_labels = lb.transform(test_labels)
    

    
    
    train_x, test_x, train_y, test_y = train_test_split(df_features.to_numpy(), df_labels, test_size=0.3)
    
    # np.savetxt("./processed_data/trainx.csv", train_x, delimiter=",")
    # np.savetxt("./processed_data/testx.csv", train_y, delimiter=",")
    # np.savetxt("./processed_data/trainy.csv", train_y, delimiter=",")
    # np.savetxt("./processed_data/testy.csv", test_y, delimiter=",")
    
    
    train_dataset = TimeSeriesDataset(train_x, train_y, seq_len)
    test_dataset = TimeSeriesDataset(test_x, test_y, seq_len)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    return train_dataloader, test_dataloader
 
    
    
features = 38
batch_size = 30
seq_len = 10
output_dim = 14

train_dataloader, test_dataloader = generate_data(features, batch_size, seq_len)

for idx,batch in enumerate(test_dataloader):
    
    print(idx, batch[0].shape, batch[1].shape, batch[2].shape)
    break

All Trials:  23
['I0' 'I1' 'I2']
0 torch.Size([30, 10, 126]) torch.Size([30, 11, 3]) torch.Size([30, 3])


## Models

LSTM Model

In [138]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        
        out, _ = self.lstm(x)

        out = self.fc(out)

        return out
    

class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]



def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

Transformer Model

In [139]:
class GlobalMaxPooling1D(nn.Module):

    def __init__(self, data_format='channels_last'):
        super(GlobalMaxPooling1D, self).__init__()
        self.data_format = data_format
        self.step_axis = 1 if self.data_format == 'channels_last' else 2

    def forward(self, input):
        return torch.max(input, axis=self.step_axis).values
    
    
class TransformerModel(nn.Module):
    
    def __init__(self, input_dim, output_dim, d_model, nhead, num_layers,  dropout=0.1):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout), num_layers=num_layers
        )
        
        self.max_pool = GlobalMaxPooling1D()
        self.fc = nn.Linear(input_dim, d_model)
        self.out = nn.Linear(d_model, output_dim)
        
    def forward(self, x):
        x = self.fc(x)
        
        x = self.max_pool(x)
  
        x = self.transformer(x)
        

        x = self.out(x)

        return x

Trans-LSTM Model

In [140]:

class TransLSTMModel(nn.Module):
    
    def __init__(self, input_dim, output_dim, d_model, nhead, num_layers, hidden_dim, layer_dim, dropout=0.1):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout), num_layers=num_layers
        )
        
        self.lstm = LSTMModel(d_model, hidden_dim, layer_dim, output_dim)
        
        self.max_pool = GlobalMaxPooling1D()
        self.fc = nn.Linear(input_dim, d_model)
        self.out = nn.Linear(d_model, output_dim)
        
    def forward(self, x):
        
        
        x = self.fc(x)
        
        
        x = self.max_pool(x)
        
        x = self.transformer(x)
    
        x = self.lstm(x)
        

        return x
    

Train Test Loops

In [141]:

def train_loop_tlstm(dataloader,model,optimizer,scheduler,criterion, epochs):
    
    model.train()
    # training loop
    for epoch in range(epochs):
        running_loss = 0.0
        for i, batch in enumerate(dataloader):

            optimizer.zero_grad()
            
            x, y, y_seq = batch
            x = x.to(torch.float32)
            y = y.to(torch.float32)
            y_seq = y_seq.to(torch.float32)
            
            x = x.cuda()
            y = y.cuda()
            y_seq = y_seq.cuda()
            
            
            y_pred = model(x)

#             print(y_pred.shape, y_seq.shape)
            
            loss = criterion(y_pred, y_seq)
            loss.backward()

            optimizer.step()
            scheduler.step()
            
            running_loss += loss.item()
            
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader):.6f}")

In [142]:



def train_loop(dataloader,model,optimizer,criterion, epochs):
    
    # training loop
    for epoch in range(epochs):
        running_loss = 0.0
        for i, batch in enumerate(dataloader):
            optimizer.zero_grad()
            x, y, y_seq = batch
            x = x.to(torch.float32)
            y = y.to(torch.float32)
            y_seq = y_seq.to(torch.float32)
            
            
            x = x.cuda()
            y = y.cuda()
            y_seq = y_seq.cuda()
        
            y_pred = model(x)

            # print(y_pred.shape, y_seq.shape)
            
            loss = criterion(y_pred, y_seq)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader):.6f}")


In [143]:
def evaluation_loop(dataloader, model,criterion):

    model.eval()

    total_accuracy = []
    for i, batch in enumerate(test_dataloader):
        x, y, y_seq = batch
        x = x.to(torch.float32)
        y = y.to(torch.float32)
        y_seq = y_seq.to(torch.float32)
        
        x = x.cuda()
        y = y.cuda()
        y_seq = y_seq.cuda()
        
        
        
        y_pred = model(x)
        
        total_inputs = 0
        true_pred = []
        
        
        for idx,y in enumerate(y_pred):
            
            total_inputs += 1
            
            output_argmax = torch.argmax(y)
            gt_argmax = torch.argmax(y_seq[idx])
        
            if(output_argmax == gt_argmax):
                true_pred.append(output_argmax)
                
            accuracy = len(true_pred)/total_inputs
            
            # print("Accuracy: ",accuracy)
            total_accuracy.append(accuracy)
            
            
                
        loss = criterion(y_pred, y_seq)
        # print(i, "Loss: ", loss)
        
    avg_accuracy = np.average(total_accuracy)
    print("Average accuracy: ", avg_accuracy)
    return avg_accuracy
    
    

Model Initiation

In [144]:
def initiate_model_tlstm(d_model, nhead, num_layers,input_dim,output_dim, hidden_dim, layer_dim, lr, iterations_per_epoch ):

    model = TransLSTMModel(input_dim=input_dim, output_dim=output_dim, d_model=d_model, nhead=nhead, num_layers=num_layers, hidden_dim=hidden_dim, layer_dim=layer_dim)
    
    model = model.cuda()
    
    optimizer = optim.Adam(model.parameters(), lr=lr) # adam
    
    sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

    criterion = nn.CrossEntropyLoss()
    
    return model, optimizer,sched, criterion

In [145]:
def initiate_model(device, d_model, nhead, num_layers, features, output_dim, lr):
    model = TransformerModel(input_dim=features, output_dim=output_dim, d_model=d_model, nhead=nhead, num_layers=num_layers)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.0001)
    criterion = nn.CrossEntropyLoss()
    
    return model, optimizer, criterion

In [146]:
batch_size = 64
seq_len = 5


epochs = 30
iterations_per_epoch = 500
lr = 1e-4

# lstm
hidden_dim = 512
layer_dim = 2
seq_dim = 128

# transformer
d_model = 128
nhead=8
num_layers=4

batch = next(iter(train_dataloader))
features = batch[0].shape[-1]
output_dim = batch[2].shape[-1]

print(features, output_dim)

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")
    
print(device)




126 3
GPU is available
cuda


Training and Testing

In [147]:
model_arch = 'transformer'
model_arch = 'translstm'

if(model_arch == 'transformer'):
    model,optimizer,criterion = initiate_model(device, d_model, nhead, num_layers, features, output_dim, lr)
    
    train_dataloader, test_dataloader = generate_data(features, batch_size, seq_len)

    train_loop(dataloader=train_dataloader, model=model, optimizer=optimizer, criterion=criterion, epochs=epochs)

    acc = evaluation_loop(dataloader=test_dataloader, model=model, criterion=criterion)

    subject_accuracy = {'accuracy':acc}
    print(subject_accuracy)
    
else:
    model,optimizer,scheduler,criterion = initiate_model_tlstm(d_model, nhead, num_layers,features,output_dim, hidden_dim, layer_dim, lr,  iterations_per_epoch=iterations_per_epoch)
    
    train_dataloader, test_dataloader = generate_data(features, batch_size, seq_len)

    train_loop_tlstm(dataloader=train_dataloader, model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion, epochs=epochs)

    acc = evaluation_loop(dataloader=test_dataloader, model=model, criterion=criterion)

    subject_accuracy = {'accuracy':acc}
    print(subject_accuracy)


All Trials:  23
['I0' 'I1' 'I2']
Epoch 1, Loss: 0.746747
Epoch 2, Loss: 0.670312
Epoch 3, Loss: 0.665763
Epoch 4, Loss: 0.652167
Epoch 5, Loss: 0.639889
Epoch 6, Loss: 0.644612
Epoch 7, Loss: 0.615627
Epoch 8, Loss: 0.594180
Epoch 9, Loss: 0.574692
Epoch 10, Loss: 0.556901
Epoch 11, Loss: 0.524874
Epoch 12, Loss: 0.489345
Epoch 13, Loss: 0.469621
Epoch 14, Loss: 0.452086
Epoch 15, Loss: 0.446329
Epoch 16, Loss: 0.437279
Epoch 17, Loss: 0.429520
Epoch 18, Loss: 0.430132
Epoch 19, Loss: 0.474947
Epoch 20, Loss: 0.464898
Epoch 21, Loss: 0.449642
Epoch 22, Loss: 0.431453
Epoch 23, Loss: 0.419836
Epoch 24, Loss: 0.409354
Epoch 25, Loss: 0.392824
Epoch 26, Loss: 0.381259
Epoch 27, Loss: 0.366344
Epoch 28, Loss: 0.352681
Epoch 29, Loss: 0.346126
Epoch 30, Loss: 0.338470
Average accuracy:  0.7680290479328891
{'accuracy': 0.7680290479328891}
